## Structured Output Perceptron

Ondrej Prenek
10.1.2019

## Assignment 1

Necessary outputs

In [25]:
import numpy as np
import scipy.io as sio

### Loading data

In [26]:
def load_data():
    mat = sio.loadmat('ocr_names.mat')
    trn_data = mat.get('TrnData')
    tst_data = mat.get('TstData')

    trn_size = trn_data.shape[1]
    trn_x = list()
    trn_y = list()
    for i in range(trn_size):
        trn_x.append(trn_data[0, i][1])
        trn_y.append(trn_data[0, i][2][0])

    tst_size = tst_data.shape[1]
    tst_x = list()
    tst_y = list()
    for i in range(tst_size):
        tst_x.append(tst_data[0, i][1])
        tst_y.append(tst_data[0, i][2][0])

    return trn_x, trn_y, tst_x, tst_y

In [27]:
trn_x, trn_y, tst_x, tst_y = load_data()

### Perceptron class

In [28]:
class MultiClassPerceptron(object):
    def __init__(self, n_letters=26, n_features=8256):
        """
        Linear (dense, fully-connected) layer.
        :param n_letters
        :param n_dims:
        """
        self.n_letters = n_letters
        self.n_features = n_features
        self.W = np.zeros(shape=(self.n_letters, self.n_features))
        self.b = np.ones(shape=(self.n_letters, 1))

    def predict(self, x):
        z = np.dot(self.W, x) + self.b
        return np.argmax(z, axis=0)

    def train(self, X, Y, labels, threshold=500):
        """
        :param X: training samples (n_samples)
        :param y: (n_samples x letters)
        :param threshold:
        """

        all_correct = False
        for i in range(threshold):
            if all_correct:
                break

            print("Epoch: " + str(i))

            err_seqs = 0
            err_chars = 0
            all_correct = True
            for x, y in zip(X, Y):
                y_pred = self.predict(x)
                y_corr = list(map(lambda c: labels.index(c), y))

                if not np.array_equal(y_pred, y_corr):
                    all_correct = False
                    err_seqs += 1

                for j in range(len(y_pred)):
                    if not y_pred[j] == y_corr[j]:
                        err_chars += 1
                        c_correct = y_corr[j]
                        self.W[c_correct] += x[:, j]
                        self.b[c_correct] += 1

                        c_incorrect = y_pred[j]
                        self.W[c_incorrect] -= x[:, j]
                        self.b[c_incorrect] -= 1

            num_of_chars = sum(map(len, Y))

            print("Char Error: " + str(err_chars / num_of_chars))
            print("Seq Error: " + str(err_seqs / len(Y)))

    def test(self, X, Y, labels):
        err_words = 0
        err_chars = 0

        for x, y in zip(X, Y):
            y_pred = self.predict(x)
            y_corr = list(map(lambda c: labels.index(c), y))

            if not np.array_equal(y_pred, y_corr):
                err_words += 1
                err_chars += len(np.nonzero(np.subtract(y_pred, y_corr))[0])

        num_of_chars = sum(map(len, Y))
        char_err = err_chars / num_of_chars
        seq_err = err_words / len(Y)

        print("--------------------------------")
        print("TEST ERROR")
        print("Seq Error: " + str(seq_err))
        print("Char Error: " + str(char_err))

### Perceptron Training

In [29]:
labels = sorted(''.join(set(''.join(trn_y))))
model = MultiClassPerceptron(n_letters=len(labels), n_features=trn_x[0].shape[0])
model.train(trn_x, trn_y, labels)

Epoch: 0
Char Error: 0.6303387334315169
Seq Error: 0.925
Epoch: 1
Char Error: 0.37825233186057927
Seq Error: 0.79
Epoch: 2
Char Error: 0.2930780559646539
Seq Error: 0.691
Epoch: 3
Char Error: 0.24104074619538537
Seq Error: 0.633
Epoch: 4
Char Error: 0.18335787923416788
Seq Error: 0.537
Epoch: 5
Char Error: 0.15463917525773196
Seq Error: 0.454
Epoch: 6
Char Error: 0.13082965144820816
Seq Error: 0.396
Epoch: 7
Char Error: 0.11266568483063329
Seq Error: 0.365
Epoch: 8
Char Error: 0.08934707903780069
Seq Error: 0.294
Epoch: 9
Char Error: 0.07216494845360824
Seq Error: 0.26
Epoch: 10
Char Error: 0.07609229258713794
Seq Error: 0.261
Epoch: 11
Char Error: 0.07241040746195386
Seq Error: 0.254
Epoch: 12
Char Error: 0.054982817869415807
Seq Error: 0.202
Epoch: 13
Char Error: 0.05326460481099656
Seq Error: 0.194
Epoch: 14
Char Error: 0.05154639175257732
Seq Error: 0.188
Epoch: 15
Char Error: 0.046146293568973984
Seq Error: 0.166
Epoch: 16
Char Error: 0.037800687285223365
Seq Error: 0.14
Epoch: 17

### Evaluating perceptron on test data

In [30]:
model.test(tst_x, tst_y, labels)

--------------------------------
TEST ERROR
Seq Error: 0.694
Char Error: 0.2502360717658168


|                                    | $R^{seq}$ (%) | $R^{char}$ (%) |
|------------------------------------|---------------|----------------|
| independent multi-class classifier | 69.4          | 25.024         |